### Trending Hashtags
Get tweets from a folder "/tmp/tweets" and find trending Hashtags

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [3]:
# create spark session
spark = SparkSession.builder\
            .appName("demo06")\
            .getOrCreate()


24/12/25 11:03:24 WARN Utils: Your hostname, nilesh-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.102 instead (on interface wlp0s20f3)
24/12/25 11:03:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 11:03:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# cmd> mkdir /tmp/tweets

dirPath = "file:///tmp/tweets"
tweetSchema = "id STRING, time STRING, text STRING"
data = spark.readStream\
            .format("json")\
            .option("path", dirPath)\
            .schema(tweetSchema)\
            .load()
data.printSchema()


root
 |-- id: string (nullable = true)
 |-- time: string (nullable = true)
 |-- text: string (nullable = true)



In [6]:
# processing -- split into words and get hashtags and count them
result = data\
            .select(explode(split(lower("text"), "[^#a-z]")).alias("word"))\
            .where("word LIKE '#%' AND word != '#'")\
            .groupBy("word").count()\
            .orderBy(desc("count")).limit(20)

In [10]:
# write streaming data frame -- sink=memory | appln runs only one time
query = result.writeStream\
    .trigger(processingTime="10 seconds")\
    .format("memory")\
    .queryName("hashtags")\
    .outputMode("complete")\
    .start()


24/12/25 11:08:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7eb0faad-3915-4da2-9a8b-64165e157e7b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/25 11:08:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
%load_ext sparksql_magic

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [22]:
%%sparksql

SELECT * FROM hashtags;

active spark session is not found


In [21]:
# destroy spark session
spark.stop()